# Giriş işlemleri

In [ ]:
%%capture
!pip install pandas numpy datasets torch transformers matplotlib
!pip install sympy --upgrade # bu kısmı colabde çalıştırmak gerekiyor eğer bir defa çalıştıysanız oturumu yeniden başlattıktan sonra bir daha çalıştırmayın

In [ ]:
import os
import pandas as pd
import numpy as np
from datasets import load_dataset
import sys
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn

In [ ]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    kok_dizin = "/content/drive/MyDrive/TurkNLP-ModelBench"
else:
    kok_dizin = os.getcwd()
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

# Veri Kümeleri Okuma

## Veri kümeleri dizini alma

In [ ]:
sys.path.append(kok_dizin)
from dizin_yardimcisi import veri_kumesi_dizin_al, veri_kumesi_adlari_listele, veri_kumesi_yukle, ShotType, shot_sonucu_kaydet, shot_sonucu_mevcut_mu, shot_sonuc_oku
veri_kumeleri_dizini = veri_kumesi_dizin_al()
print(f"Veri kümeleri dizini: {veri_kumeleri_dizini}")

## Veri kümesi adlarını alma

In [ ]:
veri_kumesi_adlari = veri_kumesi_adlari_listele(veri_kumeleri_dizini)
print(f"Veri kümeleri adları: {veri_kumesi_adlari}")

## Veri kümelerini oku

In [ ]:
sinama_kumeleri = {}
print("Sınama Veri kümeleri yükleniyor...")
for veri_kumesi_adi in veri_kumesi_adlari:
    sinama_kumeleri[veri_kumesi_adi] = veri_kumesi_yukle(veri_kumesi_adi, egitim_kumesi_mi=False, klasor_yolu=veri_kumeleri_dizini)
    print(f"{veri_kumesi_adi}: {sinama_kumeleri[veri_kumesi_adi]}")
print("Tüm Sınama veri kümeleri yüklendi.")

# Sınama İşlemleri

## Talimat oluşturma sınıfları

In [1]:
class EtiketMetni:
    _etiket_metinleri = {}
    _zero_shot_talimat_sablonu = ''
    _3_shot_talimat_sablonu = ''
    _5_shot_talimat_sablonu = ''
    _talimat = ''
    _talimat_on_eki = '### Instruction:'
    _cevap_on_eki = '### Response:'
    _ornek_girdisinin_on_eki = 'Soru::'
    _ornek_ciktisinin_on_eki = 'Cevap:'
    _ornek_girdi_metinleri = []
    _ornek_cikti_metinleri = []

    def __init_subclass__(cls, **kwargs):
        """
        Bu metot, alt sınıfların oluşturulmasını kontrol eder ve gerekli şablonları oluşturur.

        Args:
            cls: class: alt sınıf
            **kwargs: dict: keyword arguments
        Returns:
            None
        """
        super().__init_subclass__(**kwargs)
        # Zero-shot şablonunu oluştur
        cls._zero_shot_talimat_sablonu = cls.zero_shot_sablon_olustur(cls._ana_talimat)
        
        # 3-shot ve 5-shot şablonlarını oluştur
        cls._3_shot_talimat_sablonu = cls.n_shot_sablon_olustur(
            cls._ana_talimat, cls._ornek_girdi_metinleri[:3], cls._ornek_cikti_metinleri[:3]
        )
        cls._5_shot_talimat_sablonu = cls.n_shot_sablon_olustur(
            cls._ana_talimat, cls._ornek_girdi_metinleri, cls._ornek_cikti_metinleri
            )
    @classmethod
    def get(cls, talimat_sablonu: str, cumle: str, etiket_tipi: int) -> str:
        """
        Bu metot, talimat şablonunu ve cümleyi alır ve etiket tipine göre talimatı doldurur.

        Args:
            talimat_sablonu: str: talimat şablonu
            cumle: str: cümle
            etiket_tipi: int: etiket tipi

        Returns:
            str: doldurulmuş talimat
        """
        if etiket_tipi in cls._etiket_metinleri:
            return talimat_sablonu.format(cumle, cls._etiket_metinleri[etiket_tipi])
        else:
            exception_text = f"Geçersiz etiket tipi: {etiket_tipi}. Etiket tipi 0 veya 1 olmalıdır."
            raise Exception(exception_text)

    @classmethod
    def get_zero_shot(cls, cumle: str, etiket_tipi: int) -> str:
        return cls.get(cls._zero_shot_talimat_sablonu, cumle, etiket_tipi)

    @classmethod
    def get_3_shot(cls, cumle: str, etiket_tipi: int) -> str:
        return cls.get(cls._3_shot_talimat_sablonu, cumle, etiket_tipi)

    @classmethod
    def get_5_shot(cls, cumle: str, etiket_tipi: int) -> str:
        return cls.get(cls._5_shot_talimat_sablonu, cumle, etiket_tipi)
    @classmethod
    def zero_shot_sablon_olustur(cls, ana_talimat) -> str:
        return sablon_olustur(ana_talimat, cls._talimat_on_eki, cls._cevap_on_eki, cls._ornek_girdisinin_on_eki)
    @classmethod
    def n_shot_sablon_olustur(cls, ana_talimat, ornek_girdi_metinleri, ornek_cikti_metinleri) -> str:
        return sablon_olustur(ana_talimat, cls._talimat_on_eki, cls._cevap_on_eki,
                              cls._ornek_girdisinin_on_eki, cls._ornek_ciktisinin_on_eki,
                              ornek_girdi_metinleri, ornek_cikti_metinleri)


def sablon_olustur(talimat: str, talimat_on_eki: str, cevap_on_eki: str,
                   ornek_girdisinin_on_eki: str = None, ornek_ciktisinin_on_eki: str = None, 
                   ornek_girdi_metinleri: list = None, ornek_cikti_metinleri: list = None) -> str:
    """
    Bu metot, talimat şablonunu ve cümleyi alır ve etiket tipine göre talimatı doldurur.

    Args:
        talimat: kullanıcının modelden istediği işlemi belirten talimat
        talimat_on_eki: talimat metninin başına eklenen metin
        ornek_girdisinin_on_eki: örnek girdi metninin başına eklenen metin (few-shot öğrenme için)
        ornek_ciktisinin_on_eki: örnek çıktı metninin başına eklenen metin (few-shot öğrenme için)
        cevap_on_eki: cevap metninin başına eklenen metin
        ornek_girdi_metinleri: örnek girdi metinleri
        ornek_cikti_metinleri: örnek çıktı metinleri
    
    Returns:
        str: doldurulmuş talimat
    """
    sablon = f"{talimat_on_eki} {talimat}"
    if ornek_girdi_metinleri is not None and ornek_cikti_metinleri is not None:
        for ornek_girdi_metni, ornek_cikti_metni in zip(ornek_girdi_metinleri, ornek_cikti_metinleri):
            sablon += f"\n{ornek_girdisinin_on_eki} {ornek_girdi_metni}\n{ornek_ciktisinin_on_eki} {ornek_cikti_metni}"
    
    # Son olarak biçimlendirilebilir soru ve cevap alanları ekle
    sablon += f"\n{ornek_girdisinin_on_eki} {{}}\n{cevap_on_eki} {{}}"
    
    return sablon


class SST2EtiketMetni(EtiketMetni):
    _etiket_metinleri = {
        0: "Bu cümle olumsuz bir duygu ifade ediyor.",
        1: "Bu cümle olumlu bir duygu ifade ediyor."
    }
    _ornek_girdi_metinleri = [
        "Bu filmi izledikten sonra hayata bakış açım değişti, gerçekten harikaydı.",
        "Hayatımda bu kadar kötü bir hizmet görmedim, bir daha asla buraya gelmem.",
        "Kitap beni derinden etkiledi ve karakterlerle bağ kurdum.",
        "Ürün beklediğim gibi çıkmadı, hayal kırıklığına uğradım.",
        "Konser muhteşemdi, müzik ve atmosfer büyüleyiciydi."
    ]
    _ornek_cikti_metinleri = [
        _etiket_metinleri[1],
        _etiket_metinleri[0],
        _etiket_metinleri[1],
        _etiket_metinleri[0],
        _etiket_metinleri[1]
    ]
    _ana_talimat = "Aşağıdaki cümlenin duygusunu belirleyin: olumlu mu yoksa olumsuz mu?"

class COLAEtiketMetni(EtiketMetni):
    _etiket_metinleri = {
        0: "Bu cümle dilbilgisi kurallarına uygun değildir.",
        1: "Bu cümle dilbilgisi kurallarına uygundur."
    }
    _ornek_girdi_metinleri = [
        "Her sabah erken kalkıp koşuya çıkarım.",
        "Sabah erken kalkıp koşuya çıkarım yapmamıştım.",
        "Kitap okumayı ve yeni şeyler öğrenmeyi severim.",
        "Olay bu açıdan 95 derece döndürülebilir.", 
        "Buluşmaya gidiyorum çünkü arkadaşımı görmeyi özledim."
    ]
    _ornek_cikti_metinleri = [
        _etiket_metinleri[1],
        _etiket_metinleri[0],
        _etiket_metinleri[1],
        _etiket_metinleri[0],
        _etiket_metinleri[1]
    ]
    _ana_talimat = "Aşağıdaki cümlenin dilbilgisi kurallarına uygun olup olmadığını belirleyin."

## Sınama Fonksiyonları

In [ ]:
def tahmin_kontrol_et(model: AutoModel, tokenizer: AutoTokenizer, veri_satiri: pd.Series,
                      talimat_olusturma_sinifi: EtiketMetni, shot_tipi: ShotType) -> bool:
    """
    Verilen bir veri satırı için modelin doğru tahmin yapıp yapmadığını kontrol eder.
    
    Parametreler:
        model: Hugging Face model
        tokenizer: Model için tokenizer
        veri_satiri: Tek bir veri satırı (sentence ve label içermeli)
        talimat_olusturma_sinifi: Etiket metinlerini oluşturacak sınıf
        shot_tipi: ShotType Enum değeri (ZERO_SHOT, THREE_SHOT, FIVE_SHOT)
    
    Dönüş:
        bool: Doğru cevabın olasılığı en yüksekse True, değilse False
    """
    # Veri satırından metin ve gerçek etiketi al
    metin = getattr(veri_satiri, "sentence")
    gercek_etiket = getattr(veri_satiri, "label")
    
    # Mevcut etiket tiplerini al
    etiket_tipleri = list(talimat_olusturma_sinifi._etiket_metinleri.keys())
    
    en_yuksek_olasilik = float('-inf')
    en_olasi_etiket = None
    
    # Her bir etiket tipi için olasılık hesapla
    with torch.no_grad():
        for etiket in etiket_tipleri:
            # Shot tipine göre uygun talimat oluşturma metodunu seç
            if shot_tipi == ShotType.ZERO_SHOT:
                prompt = talimat_olusturma_sinifi.get_zero_shot(metin, etiket)
            elif shot_tipi == ShotType.THREE_SHOT:
                prompt = talimat_olusturma_sinifi.get_3_shot(metin, etiket)
            elif shot_tipi == ShotType.FIVE_SHOT:
                prompt = talimat_olusturma_sinifi.get_5_shot(metin, etiket)
            else:
                raise ValueError(f"Geçersiz shot tipi: {shot_tipi}")
            
            # Metni tokenize et
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)
            
            # Modelin bulunduğu cihaza gönder
            inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}
            
            # Modeli çalıştır
            outputs = model(**inputs)
            
            # Model çıktısını al (Modele bağlı olarak last_hidden_state veya logits olabilir)
            try:
                hidden_states = outputs.last_hidden_state[:, -1, :]
                # hidden_states'i bir skor vektörüne dönüştür
                # Örneğin, bir doğrusal katman kullanarak
                skor = torch.mean(hidden_states)  # Basit bir örnek
            except AttributeError:
                # Eğer last_hidden_state yoksa, model sınıflandırma modeli olabilir
                skor = outputs.logits.mean()  # Logits skorlarının ortalaması
                
            # Skoru bir olasılık değerine dönüştür (0-1 arasında)
            olasilik = torch.sigmoid(skor).item()
            
            # En yüksek olasılığı güncelle
            if olasilik > en_yuksek_olasilik:
                en_yuksek_olasilik = olasilik
                en_olasi_etiket = etiket
    
    # Modelin tahmini gerçek etiketle eşleşiyor mu?
    return en_olasi_etiket == gercek_etiket
def tek_shot_dogruluk_hesapla(model: AutoModel, tokenizer: AutoTokenizer, veri_kumesi: pd.DataFrame,
                              etiket_metni_sinifi: EtiketMetni, shot_tipi: ShotType) -> dict:
    """
    Modelin belirli bir veri kümesi üzerindeki doğruluk oranını hesaplar.
    
    Parametreler:
        model: Değerlendirilecek model
        tokenizer: Model için tokenizer
        veri_kumesi: Değerlendirme yapılacak veri kümesi
        etiket_metni_sinifi: Etiket metinlerini oluşturacak sınıf (örn. SST2EtiketMetni)
        shot_tipi: ShotType (ZERO_SHOT, THREE_SHOT, FIVE_SHOT)
    Dönüş:
        dict: JSON formatında sonuçlar içeren sözlük
    """
    # Doğruluk sayacını ve toplam örnek sayısını başlat
    dogru_tahmin_sayisi = 0
    toplam_ornek_sayisi = len(veri_kumesi)
    
    # Doğru ve yanlış indisleri tutacak listeler
    dogru_indisler = []
    yanlis_indisler = []
    
    # Her veri örneği için tahmin yap ve kontrol et
    for i, ornek in enumerate(veri_kumesi.itertuples()):
        # tahmin_kontrol_et fonksiyonu ile tahmin doğru mu kontrol et
        dogru_tahmin_mi = tahmin_kontrol_et(model, tokenizer, ornek, etiket_metni_sinifi, shot_tipi)
        
        # Eğer tahmin doğruysa sayacı artır ve doğru indisler listesine ekle
        if dogru_tahmin_mi:
            dogru_tahmin_sayisi += 1
            dogru_indisler.append(i)
        else:
            # Eğer tahmin yanlışsa, yanlış indisler listesine ekle
            yanlis_indisler.append(i)
    
    # Doğruluk oranını hesapla
    dogruluk_orani = dogru_tahmin_sayisi / toplam_ornek_sayisi
    
    # Sonuçları istenilen JSON formatında döndür
    sonuclar = {
        "accuracy": dogruluk_orani,
        "correct_fields": dogru_indisler,
        "incorrect_fields": yanlis_indisler
    }
    
    return sonuclar

def model_dogruluk_hesapla(model_adi: str, sinama_kumeleri: dict, model_dosya_adi_on_eki: str, device: str = "cuda") -> dict:
    """
    Modelin tüm veri kümeleri üzerindeki doğruluk oranını hesaplar ve dosyaya kaydeder.
    
    Parametreler:
        model_adi: Hugging Face model adı
        sinama_kumeleri: Değerlendirme yapılacak veri kümeleri sözlüğü (örn. {"cola": cola_df, "sst2": sst2_df})
        model_dosya_adi_on_eki: modelin kaydedileceği dosya adının ön eki
    
    Dönüş:
        dict: JSON formatında sonuçlar içeren sözlük
    """
    # Model ve tokenizer'ı yükle
    model = AutoModel.from_pretrained(model_adi)
    tokenizer = AutoTokenizer.from_pretrained(model_adi)
    model.to(device)
    if tokenizer.pad_token is None:
        print(f"{model_adi} modelinin tokenizer'ı pad_token'a sahip değil. EOS token'ı pad_token olarak ayarlanıyor.")
        tokenizer.pad_token = tokenizer.eos_token
    print(f"{model_adi} modeli ve tokenizer yüklendi.")
    # tüm veri kümelerini gez ve doğruluk oranlarını hesapla
    for sinama_kumesi_adi in sinama_kumeleri.keys():
        if sinama_kumesi_adi == 'cola':
            etiket_metni_sinifi = COLAEtiketMetni
        elif sinama_kumesi_adi == 'sst2':
            etiket_metni_sinifi = SST2EtiketMetni
        else:
            raise ValueError(f"Geçersiz veri kümesi adı: {sinama_kumesi_adi}")
        # tüm shot tipleri için doğruluk oranlarını hesapla ve kaydet
        for shot_tipi in ShotType:
            # shot sonuçları zaten mevcut mu diye kontrol et ve eğer mevcutsa atla
            if shot_sonucu_mevcut_mu(model_dosya_adi_on_eki, sinama_kumesi_adi, shot_tipi):
                print(f"{shot_tipi.name} shot sonuçları zaten mevcut. Atlanıyor.")
            else:
                print(f"{shot_tipi.name} shot doğruluk oranları hesaplanıyor...")
                # shot doğruluk oranlarını hesapla ve dosyaya kaydet
                shot_sonucu = tek_shot_dogruluk_hesapla(model, tokenizer, sinama_kumeleri[sinama_kumesi_adi], etiket_metni_sinifi, shot_tipi)
                print(f"{shot_tipi.name} shot doğruluk: {shot_sonucu['accuracy']:.4f}")
                shot_sonucu_kaydet(shot_sonucu, shot_tipi, model_dosya_adi_on_eki, sinama_kumesi_adi, model_adi)

In [ ]:
def get_model_dosya_adi_on_eki(model_adi: str):
    """
    Model adını kullanarak model dosya adının ön ekini döndürür.

    Parametreler:
        model_adi: Hugging Face model adı
    Dönüş:
        str: model dosya adı    
    """
    return model_adi.replace("/", "_")

## Sınama İşlemi

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Kullanılan cihaz: {device}")
modeller = [
    "malhajar/Mistral-7B-Instruct-v0.2-turkish",
    "ytu-ce-cosmos/Turkish-Llama-8b-Instruct-v0.1",
    "sayhan/Mistral-7B-Instruct-v0.2-turkish-GGUF",
]
for model in modeller:
    model_dosya_adi_on_eki = get_model_dosya_adi_on_eki(model)
    model_dogruluk_hesapla(model, sinama_kumeleri, model_dosya_adi_on_eki, device)

## Çizdirme İşlemi

In [ ]:
from cizdir import model_shot_puanlari_cizdir
# kahverengi, koyu kırmızı ve koyu yeşil
renkler = ['#8B4513', '#8B0000', '#006400']
shot_puanlari = {}
for veri_kumesi_adi in sinama_kumeleri.keys():
    veri_kumesi_shot_puanlari = []
    for model in modeller:
        model_dosya_adi_on_eki = get_model_dosya_adi_on_eki(model)
        veri_kumesi_shot_puanlari.append(shot_sonuc_oku(model_dosya_adi_on_eki, veri_kumesi_adi))
    model_shot_puanlari_cizdir(veri_kumesi_adi.capitalize(), veri_kumesi_shot_puanlari, veri_kumesi_adi, renkler = renkler)